In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# FraudFinder - Feature Engineering (batch)

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?&download_url=https://github.com/GoogleCloudPlatform/fraudfinder/raw/main/02_feature_engineering_batch.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/fraudfinder/blob/main/02_feature_engineering_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/02_feature_engineering_batch.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[FraudFinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the FraudFinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.

### Objective - Create new features and store in Feature Store

As you engineer features for model training, it's important to consider how the features are computed when making predictions with new data. For online predictions, you may have features that can be pre-computed via _batch feature engineering_. You may also features that need to be computed on-the-fly via _streaming-based feature engineering_. For these Fraudfinder labs, for computing features based on the last n _days_, you will use _batch_ feature engineering in BigQuery; for computing features based on the last n _minutes_, you will use _streaming-based_ feature engineering using Dataflow.

This notebook shows how to generate new features on bank transactions by customer and terminal over the last n days, by doing batch feature engineering in SQL with BigQuery. Then, you will create a feature store using Vertex AI Feature Store, and ingest your newly-created features from BigQuery into Vertex AI Feature Store, so that a feature store can become the single source of data for both training and model inference. 

You will also create some placeholder values for streaming-based feature engineering, which is covered in the next notebook, `03_feature_engineering_streaming.ipynb`.

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)


Steps performed in this notebook:

- Build customer and terminal-related features
- Create Feature store, entities and features
- Ingest feature values in Feature store from BigQuery table
- Read features from the feature store

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* BigQuery

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [51]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gcloud storage cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)


BUCKET_NAME          = "fraud-finder-483900-fraudfinder"
PROJECT              = "fraud-finder-483900"
REGION               = "us-central1"
ID                   = "6xkku"
FEATURESTORE_ID      = "fraudfinder_6xkku"
MODEL_NAME           = "ff_model"
ENDPOINT_NAME        = "ff_model_endpoint"
TRAINING_DS_SIZE     = "1000"



### Import libraries

In [52]:
# General
import datetime as dt
import json
import os
import random
import sys
import time
from datetime import datetime, timedelta
from typing import List, Union

# Data Engineering
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

# Vertex AI and Vertex AI Feature Store
from google.cloud import aiplatform as vertex_ai
from google.cloud import bigquery
from google.cloud.aiplatform import EntityType, Feature, Featurestore

### Define constants

In [86]:
# Define the date range of transactions for feature engineering (last 10 days up until yesterday)
YESTERDAY = datetime.today()
YEAR_MONTH_PREFIX = YESTERDAY.strftime("%Y-%m")
DATAPROCESSING_START_DATE = YESTERDAY.strftime("%Y-%m-%d")
DATAPROCESSING_END_DATE = YESTERDAY.strftime("%Y-%m-%d")

# Define BiqQuery dataset and tables to calculate features.
RAW_BQ_TRANSACTION_TABLE_URI = f"{PROJECT_ID}.tx.tx"
RAW_BQ_LABELS_TABLE_URI = f"{PROJECT_ID}.tx.txlabels"
FEATURES_BQ_TABLE_URI = f"{PROJECT_ID}.tx.wide_features_table"

# Define Vertex AI Feature store settings.
CUSTOMERS_TABLE_NAME = f"customers_{DATAPROCESSING_END_DATE.replace('-', '')}"
CUSTOMERS_BQ_TABLE_URI = f"{PROJECT_ID}.tx.{CUSTOMERS_TABLE_NAME}"
TERMINALS_TABLE_NAME = f"terminals_{DATAPROCESSING_END_DATE.replace('-', '')}"                    
TERMINALS_BQ_TABLE_URI = f"{PROJECT_ID}.tx.{TERMINALS_TABLE_NAME}"
ONLINE_STORAGE_NODES = 1
FEATURE_TIME = "feature_ts"
CUSTOMER_ENTITY_ID = "customers"
TERMINAL_ENTITY_ID = "terminals"

In [131]:
RAW_BQ_LABELS_TABLE_URI

'fraud-finder-483900.tx.txlabels'

In [130]:
RAW_BQ_LABELS_TABLE_URI

'fraud-finder-483900.tx.txlabels'

### Helpers

Define a set of helper functions to run BigQuery query and create features. 

In [79]:
def run_bq_query(sql: str, show=False) -> Union[str, pd.DataFrame]:
    """
    Run a BigQuery query and return the job ID or result as a DataFrame
    Args:
        sql: SQL query, as a string, to execute in BigQuery
        show: A flag to show query result in a Pandas Dataframe
    Returns:
        df: DataFrame of results from query,  or error, if any
    """

    bq_client = bigquery.Client()

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    result = client_result.result()
    print(f"Finished job_id: {job_id}")
    
    if show:
        df = result.to_arrow().to_pandas()
        return df

## Feature Engineering

### Define customer and terminal-related features for batch feature engineering

In this section, you will create features, based on historical customer behaviour and historical terminal activities. This features will be batch-generated using SQL in BigQuery, where the historical data is stored.

The query below will calculate 2 sets of features: 

1. **Customer-related features**: which describes the spending behaviour of customer within 1, 7 and 15 days time windows using number of transactions and average amount spent in dollars ($)

2. **Terminal-related features** which describes the risk of a given terminal to be exposed to fraudulent transactions within 1, 7 and 15 days using average number of fraudulent transactions in dollars ($), the number of transactions and risk index. One thing to note is that you will add some delay to take into account time that would pass between the time of transaction and the result of fraud investigation or customer claim.

You will use one month of transaction data starting from the end of January and going back to compute the features.

Below is the schema you should expect to see, after doing the batch feature engineering in BigQuery:

|feature_time           |customer_id| customer batch features   |
|-----------------------|-----------|---------------------------|
|2022-01-01 17:20:15 UTC|1          |(e.g., nb_tx,  avg_tx)     |
|2022-01-02 12:08:40 UTC|2          |(e.g., nb_tx,  avg_tx)     |
|2022-01-03 17:30:48 UTC|3          |(e.g., nb_tx,  avg_tx)     |


|feature_time           |terminal_id| terminal batch features|
|-----------------------|-----------|------------------------|
|2022-01-01 17:20:15 UTC|12345      |(e.g., risk_x_days)     |
|2022-01-02 12:08:40 UTC|26789      |(e.g., risk_x_days)     |
|2022-01-03 17:30:48 UTC|101112     |(e.g., risk_x_days)     |


#### Create the query to create batch features

Date settings to be used:

In [80]:
print(f"""
DATAPROCESSING_START_DATE: {DATAPROCESSING_START_DATE}
DATAPROCESSING_END_DATE: {DATAPROCESSING_END_DATE}
""")


DATAPROCESSING_START_DATE: 2026-01-12
DATAPROCESSING_END_DATE: 2026-01-12



##### Customer feature table

Customer table SQL query string:

In [81]:
create_customer_batch_features_query = f"""
CREATE OR REPLACE TABLE `{CUSTOMERS_BQ_TABLE_URI}` AS
WITH
  -- query to join labels with features -------------------------------------------------------------------------------------------
  get_raw_table AS (
  SELECT
    raw_tx.TX_TS,
    raw_tx.TX_ID,
    raw_tx.CUSTOMER_ID,
    raw_tx.TERMINAL_ID,
    raw_tx.TX_AMOUNT,
    raw_lb.TX_FRAUD
  FROM (
    SELECT
      *
    FROM
      `{RAW_BQ_TRANSACTION_TABLE_URI}`
    WHERE
      DATE(TX_TS) BETWEEN DATE_SUB("{DATAPROCESSING_END_DATE}", INTERVAL 15 DAY) AND "{DATAPROCESSING_END_DATE}"
    ) raw_tx
  LEFT JOIN 
    `{RAW_BQ_LABELS_TABLE_URI}` as raw_lb
  ON raw_tx.TX_ID = raw_lb.TX_ID),

  -- query to calculate CUSTOMER spending behaviour --------------------------------------------------------------------------------
  get_customer_spending_behaviour AS (
  SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    TX_AMOUNT,
    TX_FRAUD,
    
    # calc the number of customer tx over daily windows per customer (1, 7 and 15 days, expressed in seconds)
    COUNT(TX_FRAUD) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 86400 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_NB_TX_1DAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_NB_TX_7DAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_NB_TX_14DAY_WINDOW,
      
    # calc the customer average tx amount over daily windows per customer (1, 7 and 15 days, expressed in seconds, in dollars ($))
    AVG(TX_AMOUNT) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 86400 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,
    AVG(TX_AMOUNT) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,
    AVG(TX_AMOUNT) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_AVG_AMOUNT_14DAY_WINDOW,
  FROM get_raw_table)

# Create the table with CUSTOMER and TERMINAL features ----------------------------------------------------------------------------
SELECT
  PARSE_TIMESTAMP("%Y-%m-%d %H:%M:%S", FORMAT_TIMESTAMP("%Y-%m-%d %H:%M:%S", TX_TS, "UTC")) AS feature_ts,
  CUSTOMER_ID AS customer_id,
  CAST(CUSTOMER_ID_NB_TX_1DAY_WINDOW AS INT64) AS customer_id_nb_tx_1day_window,
  CAST(CUSTOMER_ID_NB_TX_7DAY_WINDOW AS INT64) AS customer_id_nb_tx_7day_window,
  CAST(CUSTOMER_ID_NB_TX_14DAY_WINDOW AS INT64) AS customer_id_nb_tx_14day_window,
  CAST(CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW AS FLOAT64) AS customer_id_avg_amount_1day_window,
  CAST(CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW AS FLOAT64) AS customer_id_avg_amount_7day_window,
  CAST(CUSTOMER_ID_AVG_AMOUNT_14DAY_WINDOW AS FLOAT64) AS customer_id_avg_amount_14day_window,
FROM
  get_customer_spending_behaviour
"""

##### Run the query 

You create the customer features table

In [97]:
run_bq_query(create_customer_batch_features_query)

Finished job_id: 02f59fa3-abd8-458a-b6ff-9647ce5cbecc


##### Inspect the result 

You can query some data rows to validate the result of the query

In [98]:
CUSTOMERS_BQ_TABLE_URI

'fraud-finder-483900.tx.customers_20260112'

In [99]:
run_bq_query(
    f"SELECT * FROM `{CUSTOMERS_BQ_TABLE_URI}` LIMIT 5",
    show=True
)

Finished job_id: 4304d0be-1ff0-4de2-af3f-1fd440f3e0d4


,feature_ts,customer_id,customer_id_nb_tx_1day_window,customer_id_nb_tx_7day_window,customer_id_nb_tx_14day_window,customer_id_avg_amount_1day_window,customer_id_avg_amount_7day_window,customer_id_avg_amount_14day_window
0,2025-12-30 02:19:11+00:00,0577388713591763,7,10,10,43.244286,43.370000,43.370000
1,2025-12-30 19:19:33+00:00,2075544427487721,7,13,13,39.130000,42.904615,42.904615
2,2025-12-30 05:39:45+00:00,2260249758695129,1,2,2,42.230000,36.575000,36.575000
3,2025-12-29 07:02:16+00:00,2436458327544140,3,4,4,65.233333,66.200000,66.200000
4,2025-12-28 01:39:17+00:00,2276460402269977,1,1,1,92.610000,92.610000,92.610000


##### Terminal feature table

Terminal table SQL query string:

In [100]:
create_terminal_batch_features_query = f"""
CREATE OR REPLACE TABLE `{TERMINALS_BQ_TABLE_URI}` AS
WITH
  -- query to join labels with features -------------------------------------------------------------------------------------------
  get_raw_table AS (
  SELECT
    raw_tx.TX_TS,
    raw_tx.TX_ID,
    raw_tx.CUSTOMER_ID,
    raw_tx.TERMINAL_ID,
    raw_tx.TX_AMOUNT,
    raw_lb.TX_FRAUD
  FROM (
    SELECT
      *
    FROM
      `{RAW_BQ_TRANSACTION_TABLE_URI}`
    WHERE
      DATE(TX_TS) BETWEEN DATE_SUB("{DATAPROCESSING_END_DATE}", INTERVAL 15 DAY) AND "{DATAPROCESSING_END_DATE}"
    ) raw_tx
  LEFT JOIN 
    `{RAW_BQ_LABELS_TABLE_URI}` as raw_lb
  ON raw_tx.TX_ID = raw_lb.TX_ID),

  # query to calculate TERMINAL spending behaviour --------------------------------------------------------------------------------
  get_variables_delay_window AS (
  SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    
    # calc total amount of fraudulent tx and the total number of tx over the delay period per terminal (7 days - delay, expressed in seconds)
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_DELAY,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS NB_TX_DELAY,
      
    # calc total amount of fraudulent tx and the total number of tx over the delayed window per terminal (window + 7 days - delay, expressed in seconds)
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 691200 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_1_DELAY_WINDOW,
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_7_DELAY_WINDOW,
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1814400 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_14_DELAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 691200 PRECEDING
      AND CURRENT ROW ) AS NB_TX_1_DELAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS NB_TX_7_DELAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1814400 PRECEDING
      AND CURRENT ROW ) AS NB_TX_14_DELAY_WINDOW,
  FROM get_raw_table),

  # query to calculate TERMINAL risk factors ---------------------------------------------------------------------------------------
  get_risk_factors AS (
  SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    # calculate numerator of risk index
    NB_FRAUD_1_DELAY_WINDOW - NB_FRAUD_DELAY AS TERMINAL_ID_NB_FRAUD_1DAY_WINDOW,
    NB_FRAUD_7_DELAY_WINDOW - NB_FRAUD_DELAY AS TERMINAL_ID_NB_FRAUD_7DAY_WINDOW,
    NB_FRAUD_14_DELAY_WINDOW - NB_FRAUD_DELAY AS TERMINAL_ID_NB_FRAUD_14DAY_WINDOW,
    # calculate denominator of risk index
    NB_TX_1_DELAY_WINDOW - NB_TX_DELAY AS TERMINAL_ID_NB_TX_1DAY_WINDOW,
    NB_TX_7_DELAY_WINDOW - NB_TX_DELAY AS TERMINAL_ID_NB_TX_7DAY_WINDOW,
    NB_TX_14_DELAY_WINDOW - NB_TX_DELAY AS TERMINAL_ID_NB_TX_14DAY_WINDOW,
      FROM
    get_variables_delay_window),

  # query to calculate the TERMINAL risk index -------------------------------------------------------------------------------------
  get_risk_index AS (
    SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    TERMINAL_ID_NB_TX_1DAY_WINDOW,
    TERMINAL_ID_NB_TX_7DAY_WINDOW,
    TERMINAL_ID_NB_TX_14DAY_WINDOW,
    # calculate the risk index
    (TERMINAL_ID_NB_FRAUD_1DAY_WINDOW/(TERMINAL_ID_NB_TX_1DAY_WINDOW+0.0001)) AS TERMINAL_ID_RISK_1DAY_WINDOW,
    (TERMINAL_ID_NB_FRAUD_7DAY_WINDOW/(TERMINAL_ID_NB_TX_7DAY_WINDOW+0.0001)) AS TERMINAL_ID_RISK_7DAY_WINDOW,
    (TERMINAL_ID_NB_FRAUD_14DAY_WINDOW/(TERMINAL_ID_NB_TX_14DAY_WINDOW+0.0001)) AS TERMINAL_ID_RISK_14DAY_WINDOW
    FROM get_risk_factors 
  )

# Create the table with CUSTOMER and TERMINAL features ----------------------------------------------------------------------------
SELECT
  PARSE_TIMESTAMP("%Y-%m-%d %H:%M:%S", FORMAT_TIMESTAMP("%Y-%m-%d %H:%M:%S", TX_TS, "UTC")) AS feature_ts,
  TERMINAL_ID AS terminal_id,
  CAST(TERMINAL_ID_NB_TX_1DAY_WINDOW AS INT64) AS terminal_id_nb_tx_1day_window,
  CAST(TERMINAL_ID_NB_TX_7DAY_WINDOW AS INT64) AS terminal_id_nb_tx_7day_window,
  CAST(TERMINAL_ID_NB_TX_14DAY_WINDOW AS INT64) AS terminal_id_nb_tx_14day_window,
  CAST(TERMINAL_ID_RISK_1DAY_WINDOW AS FLOAT64) AS terminal_id_risk_1day_window,
  CAST(TERMINAL_ID_RISK_7DAY_WINDOW AS FLOAT64) AS terminal_id_risk_7day_window,
  CAST(TERMINAL_ID_RISK_14DAY_WINDOW AS FLOAT64) AS terminal_id_risk_14day_window,
FROM
  get_risk_index
"""

##### Run the query 

You create the customer features table

In [101]:
run_bq_query(create_terminal_batch_features_query)

Finished job_id: 8dc34eba-741d-4ad5-9da9-b0ecb4ec7657


##### Inspect the result 

You can query some data rows to validate the result of the query

In [102]:
run_bq_query(
    f"SELECT * FROM `{TERMINALS_BQ_TABLE_URI}` LIMIT 5",
    show=True
)

Finished job_id: 9778f3af-d96d-4ad4-ad06-a6933192b2d4


,feature_ts,terminal_id,terminal_id_nb_tx_1day_window,terminal_id_nb_tx_7day_window,terminal_id_nb_tx_14day_window,terminal_id_risk_1day_window,terminal_id_risk_7day_window,terminal_id_risk_14day_window
0,2025-12-29 14:56:00+00:00,24868284,0,0,0,0.0,0.0,0.0
1,2025-12-28 00:01:21+00:00,36554560,0,0,0,0.0,0.0,0.0
2,2025-12-31 19:11:20+00:00,60846424,0,0,0,0.0,0.0,0.0
3,2025-12-28 04:48:34+00:00,15689566,0,0,0,0.0,0.0,0.0
4,2025-12-29 20:53:00+00:00,30325497,0,0,0,0.0,0.0,0.0


### Define customer and terminal-related features for _real-time_ feature engineering

To make more accurate predictions, you can also create real-time features to inspect, for example, the most recent minutes of activity for both customers and terminals.

In order to have these features available for training and inference, we first need to make sure they are defined in a BigQuery table as placeholders to be ingested into Vertex AI Feature Store.

In the query below, you initialize two sets of features for real-time feature engineering: 

1. Customer features which describes the spending behaviour of customer within 15, 30 and 60 minutes time windows using number of transactions and average amount spent in dollars ($)

2. Terminal features which describes the risk of a given terminal to be exposed to fraudulent transactions within 15, 30 and 60 minutes using average number of fraudulent transactions in dollars ($) and the number of transactions. 

To do so, you will:

- Add one column for each real time feature
- Set 0 as default values for each of them
- Update all real-time columns with default values

Then you will create the actual values for real-time feature engineering in the next `03_feature_engineering_streaming.ipynb` notebook. 

#### Define the query to initialize the real-time features.

##### Customer feature table

Customer table SQL query string:

In [103]:
initiate_real_time_customer_features_query = f"""
ALTER TABLE `{CUSTOMERS_BQ_TABLE_URI}`
ADD COLUMN customer_id_nb_tx_15min_window INT64,
ADD COLUMN customer_id_nb_tx_30min_window INT64,
ADD COLUMN customer_id_nb_tx_60min_window INT64,
ADD COLUMN customer_id_avg_amount_15min_window FLOAT64,
ADD COLUMN customer_id_avg_amount_30min_window FLOAT64,
ADD COLUMN customer_id_avg_amount_60min_window FLOAT64;

ALTER TABLE `{CUSTOMERS_BQ_TABLE_URI}`
ALTER COLUMN customer_id_nb_tx_15min_window SET DEFAULT 0,
ALTER COLUMN customer_id_nb_tx_30min_window SET DEFAULT 0,
ALTER COLUMN customer_id_nb_tx_60min_window SET DEFAULT 0,
ALTER COLUMN customer_id_avg_amount_15min_window SET DEFAULT 0,
ALTER COLUMN customer_id_avg_amount_30min_window SET DEFAULT 0,
ALTER COLUMN customer_id_avg_amount_60min_window SET DEFAULT 0;

UPDATE `{CUSTOMERS_BQ_TABLE_URI}`
SET customer_id_nb_tx_15min_window = 0,
    customer_id_nb_tx_30min_window  = 0,
    customer_id_nb_tx_60min_window  = 0, 
    customer_id_avg_amount_15min_window = 0,
    customer_id_avg_amount_30min_window  = 0,
    customer_id_avg_amount_60min_window  = 0
WHERE TRUE; 
"""

In [106]:
initiate_real_time_terminal_features_query = f"""
ALTER TABLE `{TERMINALS_BQ_TABLE_URI}`
ADD COLUMN terminal_id_nb_tx_15min_window INT64,
ADD COLUMN terminal_id_nb_tx_30min_window INT64,
ADD COLUMN terminal_id_nb_tx_60min_window INT64,
ADD COLUMN terminal_id_avg_amount_15min_window FLOAT64,
ADD COLUMN terminal_id_avg_amount_30min_window FLOAT64,
ADD COLUMN terminal_id_avg_amount_60min_window FLOAT64;

ALTER TABLE `{TERMINALS_BQ_TABLE_URI}`
ALTER COLUMN terminal_id_nb_tx_15min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_nb_tx_30min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_nb_tx_60min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_avg_amount_15min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_avg_amount_30min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_avg_amount_60min_window SET DEFAULT 0;

UPDATE `{TERMINALS_BQ_TABLE_URI}`
SET terminal_id_nb_tx_15min_window = 0,
    terminal_id_nb_tx_30min_window  = 0,
    terminal_id_nb_tx_60min_window  = 0,
    terminal_id_avg_amount_15min_window = 0,
    terminal_id_avg_amount_30min_window = 0,
    terminal_id_avg_amount_60min_window  = 0
WHERE TRUE; 
"""

#### Run the query above to initialize the real-time features.

In [107]:
for query in [initiate_real_time_customer_features_query, initiate_real_time_terminal_features_query]:
    run_bq_query(query)

Finished job_id: ba93c74c-4a28-4f2c-8368-ba66e3e289d4
Finished job_id: 020cf8e4-429c-4b27-9f2e-ef9f8e949d88


#### Inspect BigQuery features tables

**CUSTOMERS_TABLE_NAME** is the Dataset-Relative Name. Used in SELECT statements or functions where the BigQuery client already knows the Project ID.

**CUSTOMERS_BQ_TABLE_URI** (The Global Identifier). Vertex AI functions often require the URI because they need to know exactly which project to bill and where the data lives across the entire Google Cloud ecosystem.

Meanwhile, the simple TABLE_NAME is used for internal logging or simpler BigQuery Python client calls where the project is already defined in the bq_client object.

In [108]:
run_bq_query(
    f"SELECT * FROM `{CUSTOMERS_BQ_TABLE_URI}` LIMIT 5", show=True
)

Finished job_id: 58f03e25-a230-4601-a783-956f0abd8239


,feature_ts,customer_id,customer_id_nb_tx_1day_window,customer_id_nb_tx_7day_window,customer_id_nb_tx_14day_window,customer_id_avg_amount_1day_window,customer_id_avg_amount_7day_window,customer_id_avg_amount_14day_window,customer_id_nb_tx_15min_window,customer_id_nb_tx_30min_window,customer_id_nb_tx_60min_window,customer_id_avg_amount_15min_window,customer_id_avg_amount_30min_window,customer_id_avg_amount_60min_window
0,2025-12-28 18:51:56+00:00,1961810599440512,1,1,1,35.42,35.42,35.42,0,0,0,0.0,0.0,0.0
1,2025-12-29 05:02:52+00:00,5510218743715499,1,1,1,27.99,27.99,27.99,0,0,0,0.0,0.0,0.0
2,2025-12-28 08:39:06+00:00,1853038195252974,1,1,1,104.13,104.13,104.13,0,0,0,0.0,0.0,0.0
3,2025-12-28 03:22:40+00:00,1623335619864788,1,1,1,61.59,61.59,61.59,0,0,0,0.0,0.0,0.0
4,2025-12-28 00:57:04+00:00,6840273616938707,1,1,1,39.04,39.04,39.04,0,0,0,0.0,0.0,0.0


In [109]:
run_bq_query(
    f"SELECT * FROM `{TERMINALS_BQ_TABLE_URI}` LIMIT 5", show=True
)

Finished job_id: df069589-4ead-425a-8858-3444ba8fb612


,feature_ts,terminal_id,terminal_id_nb_tx_1day_window,terminal_id_nb_tx_7day_window,terminal_id_nb_tx_14day_window,terminal_id_risk_1day_window,terminal_id_risk_7day_window,terminal_id_risk_14day_window,terminal_id_nb_tx_15min_window,terminal_id_nb_tx_30min_window,terminal_id_nb_tx_60min_window,terminal_id_avg_amount_15min_window,terminal_id_avg_amount_30min_window,terminal_id_avg_amount_60min_window
0,2025-12-30 16:29:28+00:00,00064542,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
1,2025-12-28 01:54:53+00:00,00064542,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
2,2025-12-31 15:11:26+00:00,00064542,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
3,2025-12-28 21:36:32+00:00,00064542,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
4,2025-12-28 10:13:18+00:00,00064542,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0


Let's look at the final schema of the features table:

In [110]:
run_bq_query(f"""
SELECT column_name, data_type
FROM tx.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = '{CUSTOMERS_TABLE_NAME}'
""", show=True)

Finished job_id: 4d97f63a-c4c0-4ea5-a755-8ecb0efad320


,column_name,data_type
0,feature_ts,TIMESTAMP
1,customer_id,STRING
2,customer_id_nb_tx_1day_window,INT64
3,customer_id_nb_tx_7day_window,INT64
4,customer_id_nb_tx_14day_window,INT64
5,customer_id_avg_amount_1day_window,FLOAT64
6,customer_id_avg_amount_7day_window,FLOAT64
7,customer_id_avg_amount_14day_window,FLOAT64
8,customer_id_nb_tx_15min_window,INT64
9,customer_id_nb_tx_30min_window,INT64


In [111]:
run_bq_query(f"""
SELECT column_name, data_type
FROM tx.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = '{TERMINALS_TABLE_NAME}'
""", show=True)

Finished job_id: 90281ed7-0497-4e7e-9c9f-a03053fda79d


,column_name,data_type
0,feature_ts,TIMESTAMP
1,terminal_id,STRING
2,terminal_id_nb_tx_1day_window,INT64
3,terminal_id_nb_tx_7day_window,INT64
4,terminal_id_nb_tx_14day_window,INT64
5,terminal_id_risk_1day_window,FLOAT64
6,terminal_id_risk_7day_window,FLOAT64
7,terminal_id_risk_14day_window,FLOAT64
8,terminal_id_nb_tx_15min_window,INT64
9,terminal_id_nb_tx_30min_window,INT64


## Feature store for feature management

### What is a feature store?

The features generated are great examples of features that we can store in the [Vertex AI Feature Store](https://cloud.google.com/vertex-ai/docs/featurestore). This is because:

- The features are needed for real-time prediction
- feature values in a feature store can be used for both training and serving
- if needed, features can be shared with other use cases beyond fraud detection

Vertex AI Feature Store provides a centralized repository for organizing, storing, and serving ML features. Using a central featurestore enables an organization to efficiently share, discover, and re-use ML features at scale, which can increase the velocity of developing and deploying new ML applications.

### Why would you like to set up it?

So far you've built and stored features in BigQuery. 

<span style="color:red">Now, in order to predict fraud, you want to serve those features in real-time with millisecond scale latency. In particular, when the ML gateway receives a prediction request for a specific transaction (including customer, terminal, and transaction ids), the system needs to fetch the features related to that transaction and pass them as inputs to the model for online prediction. As you can imagine, an analytical data warehouse such as BigQuery is not able to provide low-latency near real-time read operations. </span>

Vertex AI Feature Store provides a managed service for low latency scalable feature serving. It also provides a centralized feature repository with easy APIs to search and discover features, as well as feature monitoring capabilities to track drift and other quality issues. 

Vertex AI Feature Store uses a time series data model to store a series of values for features, which enables Vertex AI Feature Store to maintain feature values as they change over time and to support point-in-time queries of feature values. Feature Store organizes resources hierarchically (`Featurestore -> EntityType -> Feature`) in the following order: 

- **Featurestore**: the resource to contains entities and features.
    - **EntityType**: under a Featurestore, an EntityType describes an minimal data entry.
        - **Feature**: under an EntityType, a feature is an attribute of the EntityType. 


You must create these resources before you can ingest data into a Feature Store. 

In the follow section, you will use create a feature store using Vertex AI Feature Store, and ingest data into it to be used later for training and model inference.

### Initialize Vertex AI SDK

Initialize the Vertex AI SDK to get access to Vertex AI services programmatically. 

In [112]:
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

### Create featurestore, `fraudfinder_<ID>`

A featurestore is the top-level container for entity types, features, and feature values. Typically, an organization creates one shared featurestore for feature ingestion, serving, and sharing across all teams in the organization.

Below you create a `featurestore` resources with different labels. 

In [113]:
try:
    # Checks if there is already a Featurestore
    ff_feature_store = vertex_ai.Featurestore(f"{FEATURESTORE_ID}")
    print(f"""The feature st01_exploratory_data_analysistore {FEATURESTORE_ID} already exists.""")
except:
    # Creates a Featurestore
    print(f"""Creating new feature store {FEATURESTORE_ID}.""")
    ff_feature_store = Featurestore.create(
        featurestore_id=f"{FEATURESTORE_ID}",
        online_store_fixed_node_count=ONLINE_STORAGE_NODES,
        labels={"team": "cymbal_bank", "app": "fraudfinder"},
        sync=True,
    )

The feature st01_exploratory_data_analysistore fraudfinder_6xkku already exists.


In [114]:
CUSTOMER_ENTITY_ID

'customers'

### Create the main entity types and their features

An entity type is a collection of semantically related features. You define your own entity types, based on the concepts that are relevant to your use case. 

In this case, you create `customer` and `transaction` entity types. 

#### Create the ```customer``` entity type 

In [115]:
try:
    # get entity type, if it already exists
    customer_entity_type = ff_feature_store.get_entity_type(entity_type_id=CUSTOMER_ENTITY_ID)
except:
    # else, create entity type
    customer_entity_type = ff_feature_store.create_entity_type(
        entity_type_id=CUSTOMER_ENTITY_ID, description="Customer Entity", sync=True
    )

#### Create features of the ```customer``` entity type

In [116]:
customer_feature_configs = {
    "customer_id_nb_tx_1day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last day",
        "labels": {"status": "passed"},
    },
    "customer_id_nb_tx_7day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last 7 days",
        "labels": {"status": "passed"},
    },
    "customer_id_nb_tx_14day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last 14 days",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_1day_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last day",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_7day_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 7 days",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_14day_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 14 days",
        "labels": {"status": "passed"},
    },
    "customer_id_nb_tx_15min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last 15 minutes",
        "labels": {"status": "passed"},
    },
    "customer_id_nb_tx_30min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last 30 minutes",
        "labels": {"status": "passed"},
    },
    "customer_id_nb_tx_60min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last 60 minutes",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_15min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 15 minutes",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_30min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 30 minutes",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_60min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 60 minutes",
        "labels": {"status": "passed"},
    },
}

In [ ]:
customer_feature_ids = customer_entity_type.batch_create_features(
    feature_configs=customer_feature_configs, sync=True
)

In [90]:
customer_feature_ids

resource name: projects/445617021596/locations/us-central1/featurestores/fraudfinder_6xkku/entityTypes/customers

#### Create the ```terminal``` entity type

In [91]:
try:
    # get entity type, if it already exists
    terminal_entity_type = ff_feature_store.get_entity_type(entity_type_id=TERMINAL_ENTITY_ID)
except:
    # else, create entity type
    terminal_entity_type = ff_feature_store.create_entity_type(
    entity_type_id=TERMINAL_ENTITY_ID, description="Terminal Entity", sync=True
)

Creating EntityType
Create EntityType backing LRO: projects/445617021596/locations/us-central1/featurestores/fraudfinder_6xkku/entityTypes/terminals/operations/7111976404882817024
EntityType created. Resource name: projects/445617021596/locations/us-central1/featurestores/fraudfinder_6xkku/entityTypes/terminals
To use this EntityType in another session:
entity_type = aiplatform.EntityType('projects/445617021596/locations/us-central1/featurestores/fraudfinder_6xkku/entityTypes/terminals')


#### Create features of the ```terminal``` entity type

In [92]:
terminal_feature_configs = {
    "terminal_id_nb_tx_1day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the last day",
        "labels": {"status": "passed"},
    },
    "terminal_id_nb_tx_7day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the 7 days",
        "labels": {"status": "passed"},
    },
    "terminal_id_nb_tx_14day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the 14 days",
        "labels": {"status": "passed"},
    },
    "terminal_id_risk_1day_window": {
        "value_type": "DOUBLE",
        "description": "Risk score calculated average number of frauds on the terminal in the last day",
        "labels": {"status": "passed"},
    },
    "terminal_id_risk_7day_window": {
        "value_type": "DOUBLE",
        "description": "Risk score calculated average number of frauds on the terminal in the last 7 days",
        "labels": {"status": "passed"},
    },
    "terminal_id_risk_14day_window": {
        "value_type": "DOUBLE",
        "description": "Risk score calculated average number of frauds on the terminal in the last 14 day",
        "labels": {"status": "passed"},
    },
    "terminal_id_nb_tx_15min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the last 15 minutes",
        "labels": {"status": "passed"},
    },
    "terminal_id_nb_tx_30min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the last 30 minutes",
        "labels": {"status": "passed"},
    },
    "terminal_id_nb_tx_60min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the last 60 minutes",
        "labels": {"status": "passed"},
    },
    "terminal_id_avg_amount_15min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 15 minutes",
        "labels": {"status": "passed"},
    },
    "terminal_id_avg_amount_30min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 30 minutes",
        "labels": {"status": "passed"},
    },
    "terminal_id_avg_amount_60min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 60 minutes",
        "labels": {"status": "passed"},
    },
}

In [93]:
terminal_feature_ids = terminal_entity_type.batch_create_features(
    feature_configs=terminal_feature_configs, sync=True
)

Batch creating features EntityType entityType: projects/445617021596/locations/us-central1/featurestores/fraudfinder_6xkku/entityTypes/terminals
Batch create Features EntityType entityType backing LRO: projects/445617021596/locations/us-central1/featurestores/fraudfinder_6xkku/entityTypes/terminals/operations/3462371856852451328
EntityType entityType Batch created features. Resource name: projects/445617021596/locations/us-central1/featurestores/fraudfinder_6xkku/entityTypes/terminals


### Inspect your feature store in the Vertex AI console

You can also inspect your feature store in the [Vertex AI Feature Store console](https://console.cloud.google.com/vertex-ai/features)

### Ingest feature values in Vertex AI Feature Store

Now we need to ingest the actual feature values you created in BigQuery into the Vertex AI Feature Store.

To ingest features values in Vertex AI Feature Store, you need to check the following requirements related to **Source Data format and Layout**:

- Features values have to [be stored](https://cloud.google.com/vertex-ai/docs/featurestore/source-data) in BigQuery tables or Avro and CSV files on Google Cloud Storage.
- Each imported feature entity *must* have an ID. ( in our case customer_ID, and TX_ID)
- Each feature entity can *optionally* have a timestamp, to specifying when the feature values are generated.

In the following queries, you ingest feature values from those BigQuery tables into Vertex AI Feature Store.

#### Ingest customer feature values into `customers` entity in Vertex AI Feature Store 

In the following section, you will import customer feature values into your feature store.

In [124]:
CUSTOMERS_FEATURES_IDS = [
    feature.name for feature in customer_feature_ids.list_features()
]
CUSTOMER_BQ_SOURCE_URI = f"bq://{CUSTOMERS_BQ_TABLE_URI}"
CUSTOMER_ENTITY_ID_FIELD = "customer_id"


'customer_id'

In [94]:
CUSTOMERS_FEATURES_IDS = [
    feature.name for feature in customer_feature_ids.list_features()
]
CUSTOMER_BQ_SOURCE_URI = f"bq://{CUSTOMERS_BQ_TABLE_URI}"
CUSTOMER_ENTITY_ID_FIELD = "customer_id"

In [94]:
CUSTOMERS_FEATURES_IDS = [
    feature.name for feature in customer_feature_ids.list_features()
]
CUSTOMER_BQ_SOURCE_URI = f"bq://{CUSTOMERS_BQ_TABLE_URI}"
CUSTOMER_ENTITY_ID_FIELD = "customer_id"

In [125]:
customer_entity_type.ingest_from_bq(
    feature_ids=CUSTOMERS_FEATURES_IDS,
    feature_time=FEATURE_TIME,
    bq_source_uri=CUSTOMER_BQ_SOURCE_URI,
    entity_id_field=CUSTOMER_ENTITY_ID_FIELD,
    disable_online_serving=False,
    worker_count=10,
    sync=True,
)

Importing EntityType feature values: projects/445617021596/locations/us-central1/featurestores/fraudfinder_6xkku/entityTypes/customers
Import EntityType feature values backing LRO: projects/445617021596/locations/us-central1/featurestores/fraudfinder_6xkku/entityTypes/customers/operations/7800288276056637440
EntityType feature values imported. Resource name: projects/445617021596/locations/us-central1/featurestores/fraudfinder_6xkku/entityTypes/customers


resource name: projects/445617021596/locations/us-central1/featurestores/fraudfinder_6xkku/entityTypes/customers

#### Monitor the `customer` features ingestion job in the console.


You can go to the [Feature Store Console](https://console.cloud.google.com/vertex-ai/ingestion-jobs) to view your ingestion job. 

#### Ingest terminal feature values into `terminal` entity in Vertex AI Feature Store 

In the following section, you will import terminal feature values into your feature store.

In [126]:
TERMINAL_ENTITY_ID = "terminal"
TERMINALS_FEATURES_IDS = [
    feature.name for feature in terminal_feature_ids.list_features()
]
TERMINALS_BQ_SOURCE_URI = f"bq://{TERMINALS_BQ_TABLE_URI}"
TERMINALS_ENTITY_ID_FIELD = "terminal_id"

In [127]:
terminal_entity_type.ingest_from_bq(
    feature_ids=TERMINALS_FEATURES_IDS,
    feature_time=FEATURE_TIME,
    bq_source_uri=TERMINALS_BQ_SOURCE_URI,
    entity_id_field=TERMINALS_ENTITY_ID_FIELD,
    disable_online_serving=False,
    worker_count=10,
    sync=True,
)

Importing EntityType feature values: projects/445617021596/locations/us-central1/featurestores/fraudfinder_6xkku/entityTypes/terminals
Import EntityType feature values backing LRO: projects/445617021596/locations/us-central1/featurestores/fraudfinder_6xkku/entityTypes/terminals/operations/4329138863260827648
EntityType feature values imported. Resource name: projects/445617021596/locations/us-central1/featurestores/fraudfinder_6xkku/entityTypes/terminals


resource name: projects/445617021596/locations/us-central1/featurestores/fraudfinder_6xkku/entityTypes/terminals

#### Monitor the ingestion jobs in the console.

The ingestion jobs you just created run asynchronously and they should take several minutes to complete. Please monitoring them in the [console](https://console.cloud.google.com/vertex-ai/ingestion-jobs).


### Search for feature values 
In this section, you'll run a search query on your feature store to validate that some data was ingested, as expected.

In [128]:
customer_aggregated_features = customer_entity_type.read(
    entity_ids=["1589196566013973", "5469689693941771", "1361459972478769"],
    feature_ids=CUSTOMERS_FEATURES_IDS,
)

In [129]:
customer_aggregated_features

,entity_id,customer_id_avg_amount_7day_window,customer_id_nb_tx_60min_window,customer_id_nb_tx_1day_window,customer_id_nb_tx_15min_window,customer_id_avg_amount_30min_window,customer_id_nb_tx_14day_window,customer_id_avg_amount_14day_window,customer_id_avg_amount_15min_window,customer_id_nb_tx_7day_window,customer_id_nb_tx_30min_window,customer_id_avg_amount_60min_window,customer_id_avg_amount_1day_window
0,1361459972478769,56.055556,0,3,0,0.0,9,56.055556,0.0,9,0,0.0,54.666667
1,1589196566013973,79.598571,0,2,0,0.0,14,79.598571,0.0,14,0,0.0,72.895000
2,5469689693941771,44.078750,0,4,0,0.0,16,44.078750,0.0,16,0,0.0,41.642500


### END

Now you can go to the next notebook `03_feature_engineering_streaming.ipynb`

## Clean up

In [ ]:
# ff_feature_store.delete(sync=True, force=True)